## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2024_12_16_CompareOpt_80nmTarget_round1.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [5]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [6]:
sample_table_rmse = pd.read_csv('Sample_table_2024_12_16_CompareOpt_80nmTarget_Round1_apdist.csv')
sample_table_apdist = pd.read_csv('Sample_table_2024_12_16_CompareOpt_80nmTarget_Round1_rmsedist.csv')

In [7]:
stober.count_stock_vials(sample_table_rmse, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 2}

In [8]:
sample_table_apdist['ethanol_volume'].sum()/10000

2.7669129096716643

## Load Tools

In [9]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [10]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [11]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [12]:
water_syringe.load_syringe(740, water_syringe.full_position+150)

Loaded syringe, remaining volume 740 uL


## Load ammonia syringe

In [15]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [13]:
ammonia_syringe.load_syringe(370, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 370 uL


## Load TEOS syringe

In [14]:
teos_syringe.load_syringe(690, teos_syringe.full_position+200)

Loaded syringe, remaining volume 690 uL


## load ethanol syringe

In [26]:
#jubilee.park_tool()

In [30]:
#mix_syringe.empty_position

In [15]:
mix_syringe.load_syringe(7100, mix_syringe.full_position+150)

Loaded syringe, remaining volume 7100 uL


## Load labware


In [49]:
samples_apdist = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples_apdist.manual_offset([(17.3,  173.0), (130.4, 173.6), (130.4, 103.8)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [26]:
samples_rmse = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 3)
samples_rmse.manual_offset([(158.3, 173.3), (269.9, 174.8), (270.6, 105.1)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [18]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(29.7, 262.5), (115.7, 262.9),  (116.1, 206.8)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [19]:
stocks_extra = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 5)
stocks_extra.manual_offset([(171.1, 264.1), (256.9, 265.0), (256.7, 208.5)])


New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [20]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [21]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks_apdist = [stocks_main[3], stocks_main[4], stocks_main[5], stocks_main[6], stocks_main[7], stocks_main[8]]
rinse_stocks_apdist = [stocks_main[9], stocks_main[10], stocks_main[11]]

In [22]:
ethanol_stocks_rmse = [stocks_extra[0], stocks_extra[1], stocks_extra[2], stocks_extra[3], stocks_extra[4], stocks_extra[5]]
rinse_stocks_rmse = [stocks_extra[6], stocks_extra[7], stocks_extra[8]]

## Check sample alignment

In [60]:
#jubilee.pickup_tool(mix_syringe)
for well in stocks_extra:
    jubilee.move_to(x = well.x, y = well.y)
    print(well.y)
    jubilee.move_to(z = well.top_+7)
    ans = input(f'Is the tip centered over well?')
    if ans == 'y':
        continue
    else:
        continue

264.1


Is the tip centered over well? 


264.3006231477075


Is the tip centered over well? 


264.50124629541494


...

Is the tip centered over well? 


208.00228229484696


Is the tip centered over well? 


208.20290544255442


Is the tip centered over well? 


In [34]:
jubilee.park_tool()

In [56]:
jubilee.pickup_tool(teos_syringe)
for i, row in sample_table_apdist.iterrows():
    loc = samples_apdist[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

173.0


Is the tip centered over sample A1? 


173.04285729362329


Is the tip centered over sample A2? 


173.08571458724654


...

Is the tip centered over sample C3? 
Is the tip centered over sample C4? 


145.2086700996
145.25152739322328


Is the tip centered over sample C5? 


145.29438468684654


Is the tip centered over sample C6? 


In [64]:
loc.x

130.29143531738507

In [57]:
jubilee.park_tool()


# Experiment


In [58]:
location_lookup_apdist = {row['uuid']:samples_apdist[row['well']] for i, row in sample_table_apdist.iterrows()}
location_lookup_rmse = {row['uuid']:samples_rmse[row['well']] for i, row in sample_table_rmse.iterrows()}

In [72]:
sample_table_apdist['ethanol_volume'].sum()

27669.129096716642

In [60]:
dilution_table_apdist = sample_table_apdist.copy()
dilution_table_rmse = sample_table_rmse.copy()

In [61]:
dilution_table_apdist['well'] = dilution_table_apdist['dilute_well']
dilution_table_rmse['well'] = dilution_table_rmse['dilute_well']

In [62]:
dilution_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,e208582e-4c43-433c-a80f-afdf6f316ff2,17.000000,8.500000,255.000010,1419.499990,2.232143,2.691584,0.003272,0.0,1700.0,C7,C7
1,1,eb72b1cf-3078-4701-b7ea-caf926b65285,109.727637,18.163508,127.154971,1444.953884,0.345824,17.373009,0.021306,1112.0,588.0,C8,C8
2,2,f5868166-a4f1-4a89-b04e-9a27fcf5fdcc,156.501164,62.749447,8.500000,1472.249390,0.242467,24.778590,0.030679,1288.0,412.0,D1,D1
3,3,9d4a106e-7434-4f1a-96b4-5cdd328353fc,17.000000,121.499975,8.500000,1553.000026,2.232143,2.691584,0.003366,0.0,1700.0,D2,D2
4,4,c5e36adc-8e26-4974-a080-a729a65ad94f,255.000010,8.500000,234.293984,1202.206006,0.148810,40.373762,0.048048,1447.0,253.0,D3,D3
5,5,0ded08ed-937d-4368-a29f-b7de0f5ddbd4,17.000000,53.639435,98.942080,1530.418485,2.232143,2.691584,0.003338,0.0,1700.0,D4,D4
6,6,5305d0c7-aee5-46f9-a63f-1cd42ee67d87,248.306568,145.138811,79.710672,1226.843949,0.152821,39.313999,0.047389,1440.0,260.0,D5,D5
7,7,9dc15241-60be-4ce1-86a1-52ababaf111b,80.015220,136.252721,85.400829,1398.331230,0.474240,12.668687,0.015536,894.0,806.0,D6,D6
8,8,e3bcc89d-00e3-499b-839a-17ebbb792a17,17.000000,8.500000,8.500000,1666.000001,2.232143,2.691584,0.003397,0.0,1700.0,D7,D7
9,9,589feb90-658d-45c0-b704-c1c5268b45ce,227.983181,135.714202,223.863569,1112.439048,0.166444,36.096228,0.042717,1417.0,283.0,D8,D8


In [63]:
location_lookup_dilute_apdist = {}

for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples_apdist[well]
    else:
        well_loc = None

    location_lookup_dilute_apdist[uuid_val] = well_loc

In [64]:
location_lookup_dilute_rmse = {}

for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples_rmse[well]
    else:
        well_loc = None

    location_lookup_dilute_rmse[uuid_val] = well_loc

In [69]:
dilution_table_rmse['ethanol_dilute_vol'].sum()

14639.0

## add ethanol

In [73]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ethanol_volume', ethanol_stocks_apdist[:3], stocks_usable_volume=15000, dwell_time = 10)

In [74]:

stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ethanol_volume', ethanol_stocks_rmse[:3], stocks_usable_volume=15000, dwell_time = 10)

In [75]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [76]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [77]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'water_volume', water_stocks, dwell_time = 7)

In [78]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [79]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ammonia_volume', ammonia_stocks)

In [80]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [81]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_apdist, rinse_stocks_apdist, 5)

In [82]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_rmse, rinse_stocks_rmse, 5)

## Add TEOS and mix

In [83]:
teos_start_time = time.time()

In [84]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks_apdist))

In [85]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks_rmse))

### Dilution

In [86]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

#dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_apdist, location_lookup_dilute_apdist, 'ethanol_dilute_vol', ethanol_stocks_apdist[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_apdist[uuid_val]
    dilute_well = location_lookup_dilute_apdist[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks_apdist:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks_apdist}')

    

In [87]:
# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_rmse, location_lookup_dilute_rmse, 'ethanol_dilute_vol', ethanol_stocks_rmse[4:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_rmse[uuid_val]
    dilute_well = location_lookup_dilute_rmse[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks_rmse:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks_rmse}')

In [88]:
jubilee.park_tool()

In [91]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [92]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1)

In [93]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1)

In [89]:
sample_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,fd81638a-14cb-4a0a-8556-dcd7a308c87e,85.564087,29.156944,99.069544,1486.209425,0.443485,13.547231,0.016712,946.0,754.0,A1,C7
1,1,793a471e-38ac-4081-be60-88905851c3e2,17.000000,125.486810,8.500000,1549.013191,2.232143,2.691584,0.003365,0.0,1700.0,A2,C8
2,2,26057b89-b361-498c-a63b-0447de234580,163.409560,81.450479,65.551081,1389.588881,0.232217,25.872386,0.031687,1305.0,395.0,A3,D1
3,3,a44315b0-0099-4630-a327-a22940b04fac,17.000000,82.227252,150.579789,1450.192959,2.232143,2.691584,0.003304,0.0,1700.0,A4,D2
4,4,67d341f0-6e24-4565-8491-b1e25915268e,17.000000,8.500000,255.000010,1419.499990,2.232143,2.691584,0.003272,0.0,1700.0,A5,D3
5,5,e610c151-4fb9-40da-b50b-42152154cc3e,255.000010,8.500000,255.000010,1181.499980,0.148810,40.373762,0.047903,1447.0,253.0,A6,D4
6,6,dd7d8c25-a268-4e24-acc8-1c2b445a0c04,113.587596,103.904405,122.599319,1359.908680,0.334072,17.984150,0.021911,1132.0,568.0,A7,D5
7,7,1740f218-6f41-4bfa-9f7d-b8ee9693cfea,66.193753,15.785255,36.089596,1581.931396,0.573263,10.480356,0.013095,725.0,975.0,A8,D6
8,8,02af0228-53e6-4259-988e-ecf5e64b49e6,39.423189,182.086964,151.508079,1326.981768,0.962541,6.241813,0.007583,64.0,1636.0,B1,D7
9,9,f6f7041d-0535-458a-a414-1f45eacbb474,229.388371,142.450392,64.049350,1264.111887,0.165424,36.318710,0.043975,1419.0,281.0,B3,D8


In [90]:
sample_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,e208582e-4c43-433c-a80f-afdf6f316ff2,17.000000,8.500000,255.000010,1419.499990,2.232143,2.691584,0.003272,0.0,1700.0,A1,C7
1,1,eb72b1cf-3078-4701-b7ea-caf926b65285,109.727637,18.163508,127.154971,1444.953884,0.345824,17.373009,0.021306,1112.0,588.0,A2,C8
2,2,f5868166-a4f1-4a89-b04e-9a27fcf5fdcc,156.501164,62.749447,8.500000,1472.249390,0.242467,24.778590,0.030679,1288.0,412.0,A3,D1
3,3,9d4a106e-7434-4f1a-96b4-5cdd328353fc,17.000000,121.499975,8.500000,1553.000026,2.232143,2.691584,0.003366,0.0,1700.0,A4,D2
4,4,c5e36adc-8e26-4974-a080-a729a65ad94f,255.000010,8.500000,234.293984,1202.206006,0.148810,40.373762,0.048048,1447.0,253.0,A5,D3
5,5,0ded08ed-937d-4368-a29f-b7de0f5ddbd4,17.000000,53.639435,98.942080,1530.418485,2.232143,2.691584,0.003338,0.0,1700.0,A6,D4
6,6,5305d0c7-aee5-46f9-a63f-1cd42ee67d87,248.306568,145.138811,79.710672,1226.843949,0.152821,39.313999,0.047389,1440.0,260.0,A7,D5
7,7,9dc15241-60be-4ce1-86a1-52ababaf111b,80.015220,136.252721,85.400829,1398.331230,0.474240,12.668687,0.015536,894.0,806.0,A8,D6
8,8,e3bcc89d-00e3-499b-839a-17ebbb792a17,17.000000,8.500000,8.500000,1666.000001,2.232143,2.691584,0.003397,0.0,1700.0,B1,D7
9,9,589feb90-658d-45c0-b704-c1c5268b45ce,227.983181,135.714202,223.863569,1112.439048,0.166444,36.096228,0.042717,1417.0,283.0,B3,D8


In [151]:
dilution_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,bbd1bfff-7563-4360-b5b8-86a1deb3928f,91.455773,121.416734,255.000010,1232.127483,0.414916,14.480053,0.017314,995.0,705.0,C7,C7
1,1,5b9ac046-6ca6-4ebc-87f3-e272edaedb7b,140.351853,8.500000,255.000010,1296.148137,0.270366,22.221694,0.026673,1240.0,460.0,C8,C8
2,2,9f782755-5ca4-4466-a1f1-a87ef665ed22,255.000010,42.990994,255.000010,1147.008985,0.148810,40.373762,0.047775,1447.0,253.0,D1,D1
3,3,f8b2207f-b63c-44d9-8fe4-70639fdd3f5c,255.000010,53.976205,8.500000,1382.523785,0.148810,40.373762,0.049511,1447.0,253.0,D2,D2
4,4,61431cc9-9557-45af-82c4-9e7acb251717,78.545307,77.696470,193.651897,1350.106326,0.483115,12.435958,0.015077,879.0,821.0,D3,D3
5,5,e773e91e-cdd8-466c-be37-8ff86cb7f610,52.806463,59.674126,39.147849,1548.371562,0.718594,8.360767,0.010419,478.0,1222.0,D4,D4
6,6,4559d494-fcc2-4dfc-a4cf-a14d93eb0029,102.107158,177.169546,8.500000,1412.223295,0.371633,16.166470,0.019945,1068.0,632.0,D5,D5
7,7,78e5ec30-0b59-4536-a9e0-21b4bac433a8,67.497684,128.016714,8.500000,1495.985602,0.562189,10.686805,0.013285,744.0,956.0,D6,D6
8,8,e42bc4d2-0157-425c-aed0-4cdec425d9ab,120.342861,146.735320,171.336467,1261.585352,0.315319,19.053701,0.022952,1164.0,536.0,D7,D7
9,9,4133ded3-a2c0-4d14-b4fb-578358436a04,17.000000,159.878042,8.500000,1514.621958,2.232143,2.691584,0.003355,0.0,1700.0,D8,D8
